In [1]:
import pandas as pd
import random
from nltk.tokenize import TweetTokenizer
import string
from collections import defaultdict
from sklearn import linear_model
from nltk.corpus import stopwords
import nltk
from sklearn import svm

In [5]:
file = pd.read_csv("./Desktop/data.csv", encoding='ISO-8859-1')
data = file.values.tolist()

master_data = []
for l in data:
    master_data.append([l[0], l[5]])
random.shuffle(master_data)

smaller_data_punc = []
smaller_data = []
# without punctuation 
punctuation = set(string.punctuation)
counter = 0
for l in master_data[:300000]:
    r = ''.join([c for c in l[1].lower() if not c in punctuation and c !='@'])
    if(l[0] == 0):
        smaller_data.append([0, r])
    else:
        smaller_data.append([1, r])
    counter +=1
# with punctuation 
counter = 0
for l in master_data[:300000]:
    r = ''.join([c for c in l[1].lower() if c !='@'])
    if(l[0] == 0):
        smaller_data_punc.append([0, r])
    else:
        smaller_data_punc.append([1, r])
    counter +=1
    
train_text = [x[1] for x in smaller_data[:200000]]

wordCount = defaultdict(int)
for l in train_text:
    l = l.split()
    if(len(l) == 0):
        continue
    wordCount[l[0]] += 1
    for i in range(1, len(l)):
        wordCount[l[i]] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:2000]]


unigram_frequency = defaultdict(float)
total_count = sum([i[0] for i in counts[:2000]])
for i in counts[:2000]:
    unigram_frequency[i[1]] = i[0]/total_count 
    
tweet = TweetTokenizer()


wordCountP = defaultdict(int)
for d in smaller_data_punc:
    review = d[1]
    tokens = tweet.tokenize(review)
    for w in tokens:
        wordCountP[w] += 1

countsP = [(wordCountP[w], w) for w in wordCountP]
countsP.sort()
countsP.reverse()

wordsP = [x[1] for x in countsP[:2000]]

wordIdP = dict(zip(wordsP, range(len(wordsP))))
wordSetP = set(wordsP)


# Calculating the most 2000 frequenct words frequncy in probalility in positive sentiment text
unigram_frequency_pos = defaultdict(float)
train_text_pos = [x[1] for x in smaller_data[:200000] if x[0] == 1]
for l in train_text_pos:
    l = l.split()
    if(len(l) == 0):
        continue
    for w in l:
        if w in words:
            unigram_frequency_pos[w] += 1


total_count = 0
for w in unigram_frequency_pos:
    total_count += unigram_frequency_pos[w]
for w in unigram_frequency_pos:
    unigram_frequency_pos[w] = unigram_frequency_pos[w]/total_count
positive_words = []
for w in unigram_frequency:
    positive_words.append([unigram_frequency_pos[w] - unigram_frequency[w], w])
positive_words.sort()
positive_words.reverse()
pos_words = [x[1] for x in positive_words][:200]


unigram_frequency_neg = defaultdict(float)
train_text_neg = [x[1] for x in smaller_data[:200000] if x[0] == 0]
for l in train_text_neg:
    l = l.split()
    if(len(l) == 0):
        continue
    for w in l:
        if w in words:
            unigram_frequency_neg[w] += 1
total_count = 0
for w in unigram_frequency_neg:
    total_count += unigram_frequency_neg[w]
for w in unigram_frequency_neg:
    unigram_frequency_neg[w] = unigram_frequency_neg[w]/total_count
negative_words = []
for w in unigram_frequency:
    negative_words.append([unigram_frequency_neg[w] - unigram_frequency[w], w])
negative_words.sort()
negative_words.reverse()
neg_words = [x[1] for x in negative_words][:200]

In [6]:
# Baseline Model
def baseFeature(datum):
    positive_count = 0
    negative_count = 0
    r = ''.join([c for c in datum[1].lower() if not c in punctuation])
    for w in r.split():
        if w in pos_words:
            positive_count += 1
        if w in neg_words:
            negative_count += 1
    if(negative_count > positive_count):
        return 0
    else:
        return 1

In [8]:
def predict_baseline(str_1):
    base_feature = baseFeature(str_1)
    if base_feature == 1:
        print("It's positive")
    else:
        print("It's negative")

In [9]:
# using unigram features with punctuation 
def feature3(datum):
    feat = [0]*len(wordsP)
    review = datum[1]
    tokens = tweet.tokenize(review)
    for w in tokens:
        if w in wordsP:
            feat[wordIdP[w]] += 1
    feat.append(1) #offset
    return feat
X = [feature3(d) for d in smaller_data_punc]
y = [d[0] for d in smaller_data_punc]
clf = svm.LinearSVC(C=10,  max_iter=5000)
clf.fit(X, y)

LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=5000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [34]:
def predict_svm(str_1):
    datum = [0,str_1]
    feat = []
    feat.append(feature3(datum))
    if clf.predict(feat)[0] == 0:
        print("It's negative")
    else:
        print("It's positive")
    